# troubleshooting pubchem bioassay cleaning
__rationale__: I am trying to reproduce Helal, Kazi Yasin, et al. JCIM 56.2 (2016): 390-398. I've downloaded the pubchem records they used in their analysis, and now I have to make sure the data are suitable to be analyzed by cleaning the associated files.

__notes on pubchem recrods__:
* first 4x rows lack activity data - are annotations for the data contained in the file
* I should dump all useless records - internal IDs for each institution, blank rows, etc. - I need to narrow down in on the bare minimum variables I need to get the analysis done
* CIDs need to be converted from float to int if that's not done already in the file
* some of them have replicates, which will need to be averaged later on - probably on the processing step
* activity data has a unique column name on each record - yeesh - how do I located this in records? Also, some screens were done with multiple replicates, so in those files, the data will have to be averaged together ...
* PUBCHEM_ACTIVITY_SCORE is a user-defined metric between 100 and 0 that allows users to prioritize hits - don't use this for your z-score calculation, because it seems to be extremely subjective and therefore variable between records
* the same compounds are screened multiple times in one screen - therefore, when calculating z scores later on, I need to account for 

__ideas on how to find activity data__
* can't do certain float - stdev, z-scores, etc. would come down
* can't do name, because the name varies with each and every assay - no consistencies between records
* can't do result description, because while some are the name of the active column, some are multi-line descriptions of the data
* can't do the XML metadata, because there's no explicit "active column" tag, just the description data that's in the flat CSV file
* ... maybe I can do the column after the PUBCHEM_ASSAYDATA_COMMENT - so far, all of the assays I've checked have had either (a) the activity data or (b) the first replicate appear after this column

# pick up here - need to figure out how to parse out the activity data
# I think 
# that the column after the comment tag above might be the consistent organization in the data -> np.where, then i + 1

__approach__:
* manually look through select files over the course of the time period to see if the formatting is consistent between files
* remove the first rows which lack information on compound activity
* ensure that the data type for 

---
## troubleshooting CID extraction and counting

In [ ]:
import os
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# import, skip desc rows
p = '../data/1-raw/public-domain-fingerprints/'
aid1 = pd.read_csv(p + '2130.csv', skiprows=[1,2,3,4])
aid1

In [ ]:
# extract CIDs, convert to ints, count
aid1['PUBCHEM_CID'].astype(int).value_counts()

hmmm ... how should we deal with the fact that some compounds are screened more than once?

Right now, having unique counts isn't the point ... we don't want to know the absolute number of times a compound has been screened - we want to know how many assays the compound has been screened in. Therfore, this information can be dropped here

... however, this will have to be accounted for later on - during the Z-score step, I'm going to have to average values for a given record. I'm noting this at the top of the page.

In [ ]:
inter_dict = {'PUBCHEM_CID':aid1['PUBCHEM_CID'].astype(int).unique(),
              'current_count':1}
intermediate_CID = pd.DataFrame(inter_dict)
intermediate_CID

In [ ]:
# start df to hold data over loops
cids = pd.DataFrame(columns=['PUBCHEM_CID', 'assay_count'])

# merge old and new
cids = pd.merge(cids, intermediate_CID, on='PUBCHEM_CID', how='outer')

# fill NaN with 0 to allow proper addition
cids = cids.fillna(0)

# add and drop the current count column
cids['assay_count'] = cids['assay_count'] + cids['current_count']
cids = cids.drop('current_count', axis=1)
cids

In [ ]:
# trying to merge second column by hand:

p = '../data/1-raw/public-domain-fingerprints/'
aid1 = pd.read_csv(p + '435005.csv', skiprows=[1,2,3,4])

aid1['PUBCHEM_CID'].astype(int).value_counts()

inter_dict = {'PUBCHEM_CID':aid1['PUBCHEM_CID'].astype(int).unique(),
              'current_count':1}
intermediate_CID = pd.DataFrame(inter_dict)


cids = pd.merge(cids, intermediate_CID, on='PUBCHEM_CID', how='outer')

cids = cids.fillna(0)

cids['assay_count'] = cids['assay_count'] + cids['current_count']
cids.drop('current_count', axis=1)

that should do the trick!

let's write the loop and run a pilot on a couple AIDs

In [ ]:
# set up imports and initialize dataframe to hold data
p = '../data/1-raw/public-domain-fingerprints/'
aid_list = ['1511.csv', '2029.csv', '493036.csv', '602162.csv', '720543.csv']

cids = pd.DataFrame(columns=['PUBCHEM_CID', 'assay_count'])
aid_count = 1

# loop over data to import, process, and add to the cids DF
for f in aid_list:
    # import and skip over desc rows
    aid = pd.read_csv(p + f, skiprows=[1,2,3,4])
    
    # correct CID data type, isolate unique CIDs
    inter_dict = {'PUBCHEM_CID':aid['PUBCHEM_CID'].astype(int).unique(),
              'current_count':1}
    intermediate_CID = pd.DataFrame(inter_dict)
    
    # merge with cids DF
    cids = pd.merge(cids, intermediate_CID, on='PUBCHEM_CID', how='outer')

    # add records and clean merged df
    cids = cids.fillna(0)
    cids['assay_count'] = cids['assay_count'] + cids['current_count']
    cids = cids.drop('current_count', axis=1)
    
    print('{count} of {total} AIDs processed'.format(count=aid_count,
                                                     total=len(aid_list)))
    aid_count += 1

cids

 yikes - missing values in the initial DFs ... let's see what's giving us the error

In [ ]:
# import, skip desc rows
p = '../data/1-raw/public-domain-fingerprints/'
aid1 = pd.read_csv(p + '602162.csv', skiprows=[1,2,3,4])
aid1

In [ ]:
aid1['PUBCHEM_CID'].loc[aid1['PUBCHEM_CID'].isna()]

In [ ]:
aid1.loc[348664]

In [ ]:
aid1['PUBCHEM_CID'].dropna()

whoops - looks just like there's an issue here - let's get rid of all the records like this

In [ ]:
# set up imports and initialize dataframe to hold data
p = '../data/1-raw/public-domain-fingerprints/'
aid_list = ['1511.csv', '2029.csv', '493036.csv', '602162.csv', '720543.csv']

cids = pd.DataFrame(columns=['PUBCHEM_CID', 'assay_count'])
aid_count = 1

# loop over data to import, process, and add to the cids DF
for f in aid_list:
    # import and skip over desc rows
    aid = pd.read_csv(p + f, skiprows=[1,2,3])
    
    # correct CID data type, isolate unique CIDs
    cid_series = aid['PUBCHEM_CID'].dropna()
    inter_dict = {'PUBCHEM_CID':cid_series.astype(int).unique(),
              'current_count':1}
    intermediate_CID = pd.DataFrame(inter_dict)
    
    # merge with cids DF
    cids = pd.merge(cids, intermediate_CID, on='PUBCHEM_CID', how='outer')

    # add records and clean merged df
    cids = cids.fillna(0)
    cids['assay_count'] = cids['assay_count'] + cids['current_count']
    cids = cids.drop('current_count', axis=1)
    
    print('{count} of {total} AIDs processed'.format(count=aid_count,
                                                     total=len(aid_list)))
    aid_count += 1

cids

In [ ]:
# testing on a different data set to manually curate
# set up imports and initialize dataframe to hold data
p = '../data/1-raw/public-domain-fingerprints/'
aid_list = ['1511.csv', '1554.csv', '1662.csv', '1663.csv', '1813.csv']

cids = pd.DataFrame(columns=['PUBCHEM_CID', 'assay_count'])
aid_count = 1

# loop over data to import, process, and add to the cids DF
for f in aid_list:
    # import and skip over desc rows
    aid = pd.read_csv(p + f, skiprows=[1,2,3])
    
    # correct CID data type, isolate unique CIDs
    cid_series = aid['PUBCHEM_CID'].dropna()
    inter_dict = {'PUBCHEM_CID':cid_series.astype(int).unique(),
              'current_count':1}
    intermediate_CID = pd.DataFrame(inter_dict)
    
    # merge with cids DF
    cids = pd.merge(cids, intermediate_CID, on='PUBCHEM_CID', how='outer')

    # add records and clean merged df
    cids = cids.fillna(0)
    cids['assay_count'] = cids['assay_count'] + cids['current_count']
    cids = cids.drop('current_count', axis=1)
    
    print('{count} of {total} AIDs processed'.format(count=aid_count,
                                                     total=len(aid_list)))
    aid_count += 1

cids

In [ ]:
cids.info()

these check out by hand!

It's remarkable that the screening sets that went into each of these initial assays are so similar, but based on the ones I tested, the compounds coming up in 5/5 or 4/5 all came up in their predicted counts when I went in and searched these files by hand